Code to parse the axon data into the sampling time of the camera

In [2]:
%pylab

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [72]:
def get_axon_signals(filename):
    """map the axon signals into a dictionary"""
    from neo.io.axonio import AxonIO
    reader = AxonIO(filename=filename)
    blocks = reader.read()
    header = reader.read_header()
    channel_names = [info['ADCChNames'] for info in header['listADCInfo']]
    channel_units = [info['ADCChUnits'] for info in header['listADCInfo']]
    times = blocks[0].segments[0].analogsignals[0].times
    sampling_period = blocks[0].segments[0].analogsignals[0].sampling_period
    times.units = 's'
    sampling_period.units = 's'
    times = np.array(times)
    sampling_period = np.float64(sampling_period)
    signals = [np.array(x) for x in blocks[0].segments[0].analogsignals]
    signals = dict(zip(channel_names,signals))
    signals['times'] = times
    return signals

def idx_by_thresh(signal,thresh = 0.1):
    """returns a lists of arrays, each array holds the indexes where sig
    is greater than thresh"""
    diffs = np.diff((signal > 1).astype(int))
    starts = np.where(diffs == 1)[0] # indexes where signal goes above thresh
    stops = np.where(diffs == -1)[0] # indexes wehre signal goes below thresh
    return [np.arange(sta+1,stp) for sta,stp in zip(starts,stops)]

def downsample_to_frames(sigs,sync_sig_name = 'CamSync'):
    """downsample the axon data into frame time"""
    exposures = idx_by_thresh(sigs[sync_sig_name],thresh = 1)
    #idx of each frame in the axon array.
    downsampled = {}
    for key in sigs.keys():
        sig = sigs[key]camera
        downsamp = np.array([np.mean(sig[ex]) for ex in exposures])
        downsampled[key] = downsamp
    return downsampled

In [3]:
#load the muscle data
import cPickle
with open('/home/psilentp/Dropbox/imaging_cycle/Fly1003/images/model_fits.cpkl') as f:
    muscle_data = cPickle.load(f)

In [5]:
# load the axon data
axon_sigs = get_axon_signals('/home/psilentp/Dropbox/imaging_cycle/Fly1003/16712002.abf')

In [71]:
# downsample into the imaging timebase (maybe ok to assume regular samples)
downsamp_sigs = downsample_to_frames(axon_sigs)